In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# task가 중첩되는 경우 실행 결과는?

import asyncio

async def inner():
    print("Inner start")
    await asyncio.sleep(1)
    print("Inner end")

async def outer():
    print("Outer start")
    task = asyncio.create_task(inner())
    print("Outer end")
    await task 

asyncio.run(outer())

Outer start
Outer end
Inner start
Inner end


In [3]:
import asyncio
import concurrent.futures
import time

def cpu_bound_task(name):
    print(f"{name}: Start heavy CPU work")
    count = 0
    for _ in range(10 ** 7):
        count += 1
    print(f"{name}: End heavy CPU work")
    return count

async def io_bound_task(name):
    print(f"{name}: Start I/O task")
    await asyncio.sleep(2)
    print(f"{name}: End I/O task")

async def main(executor):
    loop = asyncio.get_running_loop()

    # CPU 바운드 작업 2개 + I/O 바운드 작업 2개 실행
    tasks = [
        loop.run_in_executor(executor, cpu_bound_task, "CPU-1"),
        loop.run_in_executor(executor, cpu_bound_task, "CPU-2"),
        asyncio.create_task(io_bound_task("IO-1")),
        asyncio.create_task(io_bound_task("IO-2"))
    ]

    start = time.time()
    results = await asyncio.gather(*tasks)
    end = time.time()

    print(f"Results: {results}")
    print(f"Total time: {end - start:.2f} seconds")

if __name__ == "__main__":
    # print("=== ThreadPoolExecutor ===")
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #     asyncio.run(main(executor))

    print("\n=== ProcessPoolExecutor ===")
    with concurrent.futures.ProcessPoolExecutor() as executor:
        asyncio.run(main(executor))


=== ProcessPoolExecutor ===
IO-1: Start I/O task
IO-2: Start I/O task


Process SpawnProcess-1:
Process SpawnProcess-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/concurrent/futures/process.py", line 241, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/multiprocessing/queues.py", line 120, in get
    return _ForkingPickler.loads(res)
           ~~~~~~~~~~~~~~~~~~~~~^^^^^
AttributeError: Can't get attribute 'cpu_bound_task' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/multiprocessin

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

IO-1: End I/O task
IO-2: End I/O task


In [5]:
import asyncio
import concurrent.futures

def cpu_bound_task():
    print("Start heavy CPU work")
    count = 0
    for _ in range(10 ** 7):
        count += 1
    print("End heavy CPU work")
    return count

async def main():
    loop = asyncio.get_running_loop()

    # CPU 바운드 작업을 ProcessPoolExecutor 로 분리
    with concurrent.futures.ProcessPoolExecutor() as pool:
        result = await loop.run_in_executor(pool, cpu_bound_task)
        print(f"Result: {result}")

if __name__ == '__main__':
    asyncio.run(main())

Process SpawnProcess-4:
Traceback (most recent call last):
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/concurrent/futures/process.py", line 241, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/kihoon/.pyenv/versions/3.13.0/lib/python3.13/multiprocessing/queues.py", line 120, in get
    return _ForkingPickler.loads(res)
           ~~~~~~~~~~~~~~~~~~~~~^^^^^
AttributeError: Can't get attribute 'cpu_bound_task' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.